##### Master Degree in Computer Science and Data Science for Economics

# Example of biases in data classification

### Alfio Ferrara

In [1]:
import os
import numpy as np 
import pandas as pd

In [2]:
data = pd.read_excel(os.path.join("../data", "biased-cv-doc.xlsx"), index_col=0)
data.head(10)

,text,target,class_name
0,The candidate is a female coming from usa and ...,2,highly suitable
1,The candidate is a female coming from italy an...,0,not suitable
2,The candidate is a male coming from france and...,2,highly suitable
3,The candidate is a male coming from italy and ...,0,not suitable
4,The candidate is a male coming from italy and ...,0,not suitable
5,The candidate is a female coming from italy an...,0,not suitable
6,The candidate is a male coming from italy and ...,0,not suitable
7,The candidate is a male coming from germany an...,2,highly suitable
8,The candidate is a male coming from spain and ...,1,moderately suitable
9,The candidate is a female coming from usa and ...,1,moderately suitable


In [4]:
print(data.loc[0].text)

The candidate is a female coming from usa and has 2 years of esperience in the field.
    The language proficiency level is beginner. The technical proficiency level is proficient.


In [5]:
data.class_name.unique()

array(['highly suitable', 'not suitable', 'moderately suitable'],
      dtype=object)

The model trained for this example can be found in [bert_classifier](./nlp/bert_classifier.py). Use it to train the model and save the outcome on you local folders.

In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Substitute with your path
MODEL_PATH = "/Users/Flint/Data/einaudidh/bert_text_classifier"
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
model = BertForSequenceClassification.from_pretrained(MODEL_PATH, local_files_only=True)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

# 2. Funzione per fare previsioni
def predict(texts, model, tokenizer, device, max_len=256):
    model.eval()
    predictions = []
    with torch.no_grad():
        for text in texts:
            encoding = tokenizer(
                text,
                max_length=max_len,
                padding='max_length',
                truncation=True,
                return_tensors="pt"
            )
            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.append(preds.item())
    return predictions


In [7]:
sample = data['text'].values
expected_label = data['target'].values
data_to_predict = sample[:6]
y_true = expected_label[:6]

predicted_labels = predict(data_to_predict, model, tokenizer, device)

for i, (text, label) in enumerate(zip(data_to_predict, predicted_labels)):
    yt = y_true[i]
    print(f"Text: {text}\nPredicted Label: {label}\nTrue Label: {yt}\n")


Text: The candidate is a female coming from usa and has 2 years of esperience in the field.
    The language proficiency level is beginner. The technical proficiency level is proficient.
Predicted Label: 2
True Label: 2

Text: The candidate is a female coming from italy and has 2 years of esperience in the field.
    The language proficiency level is proficient. The technical proficiency level is beginner.
Predicted Label: 0
True Label: 0

Text: The candidate is a male coming from france and has 4 years of esperience in the field.
    The language proficiency level is intermediate. The technical proficiency level is proficient.
Predicted Label: 2
True Label: 2

Text: The candidate is a male coming from italy and has 4 years of esperience in the field.
    The language proficiency level is upper-intermediate. The technical proficiency level is intermediate.
Predicted Label: 0
True Label: 0

Text: The candidate is a male coming from italy and has 5 years of esperience in the field.
    T

## Stats

In [8]:
from sklearn.metrics import classification_report

In [9]:
sample = data['text'].values 
y_true = data['target'].values 
y_pred = predict(sample, model, tokenizer, device)
print(classification_report(y_true, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90       337
           1       0.68      0.70      0.69       334
           2       0.75      0.55      0.64       329

    accuracy                           0.75      1000
   macro avg       0.75      0.75      0.74      1000
weighted avg       0.75      0.75      0.74      1000



### Analysis of outcome

In [10]:
from collections import defaultdict

In [11]:
tab = pd.read_excel(os.path.join("../data", "biased-cv-tab.xlsx"), index_col=0)
tab.head(2)

,gender,country,lang,tech,target
0,female,usa,beginner,proficient,highly suitable
1,female,italy,proficient,beginner,not suitable


In [12]:
data.head(2)

,text,target,class_name
0,The candidate is a female coming from usa and ...,2,highly suitable
1,The candidate is a female coming from italy an...,0,not suitable


In [13]:
genders = defaultdict(lambda: defaultdict(lambda: 0))
country = defaultdict(lambda: defaultdict(lambda: 0))
lang = defaultdict(lambda: defaultdict(lambda: 0))
tech = defaultdict(lambda: defaultdict(lambda: 0))
stats = [genders, country, lang, tech]

for i, pred in enumerate(y_pred):
    record = tab.iloc[i]
    for j, x in enumerate(record.values[:-1]):
        stats[j][pred][x] += 1
S = [pd.DataFrame(x) for x in stats]

In [14]:
def show(df):
    return np.round(df.fillna(0) / df.sum(axis=0), 2) * 100

#### Let's check for different features and classes, where:

**Target**: 0 => not suitable, 1 => moderately suitable, 2 => highly suitable

**Features**: 0 => gender, 1 => country, 2 => language, 3 => tech

In [18]:
target = 0
feature = 1
print(len([x for x in y_pred if x == target]) / len(y_pred))
print(round(100 * S[feature] / S[feature].sum(axis=0), 2))
print(round(100 * S[feature].T / S[feature].T.sum(axis=0), 2))

0.413
             2      0      1
usa      24.28    NaN  23.55
france   27.16    NaN  27.33
spain    25.10    NaN  26.45
germany  23.46    NaN  22.67
italy      NaN  100.0    NaN
     usa  france  spain  germany  italy
2  42.14   41.25  40.13    42.22    NaN
0    NaN     NaN    NaN      NaN  100.0
1  57.86   58.75  59.87    57.78    NaN


In [23]:
show(S[1])

,2,0,1
usa,24.0,0.0,24.0
france,27.0,0.0,27.0
spain,25.0,0.0,26.0
germany,23.0,0.0,23.0
italy,0.0,100.0,0.0


In [24]:
show(S[2])

,2,0,1
beginner,23.0,25.0,30.0
intermediate,27.0,27.0,22.0
proficient,25.0,26.0,21.0
upper-intermediate,24.0,22.0,27.0


In [25]:
show(S[3])

,2,0,1
proficient,78.0,13.0,0.0
beginner,19.0,45.0,0.0
upper-intermediate,3.0,18.0,58.0
intermediate,0.0,24.0,42.0


## Born

In [31]:
from bornrule import BornClassifier
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
documents = data.text.values
vectorizer = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)
X = vectorizer.fit_transform(documents)
y = data.target.values

In [33]:
born = BornClassifier()

In [34]:
born.fit(X, y)
b_pred = born.predict(X)

/Users/Flint/.pyenv/versions/nlp/lib/python3.10/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [35]:
print(classification_report(y_true, b_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90       337
           1       0.66      0.39      0.49       334
           2       0.58      0.69      0.63       329

    accuracy                           0.69      1000
   macro avg       0.69      0.69      0.67      1000
weighted avg       0.69      0.69      0.67      1000



In [36]:
features = vectorizer.get_feature_names_out()
E = pd.DataFrame(born.explain().toarray(), index=features)

In [39]:
E.sort_values(by=0, ascending=False).head(10)

,0,1,2
italy,0.078608,0.027204,0.026003
beginner,0.011133,0.007800,0.007180
proficient,0.006782,0.006532,0.009688
upper-intermediate,0.003173,0.004392,0.003928
intermediate,0.001825,0.001994,0.001597
1,0.001494,0.001430,0.001818
2,0.000662,0.000649,0.000570
5,0.000506,0.000446,0.000456
4,0.000373,0.000417,0.000378
female,0.000222,0.000207,0.000219
